This is the test file for Porter's new models

To use it, change the modelPath variable to the path to the new model

Currently, it is configured for a model output of all letters, A-Z. If the model changes to exclude J and Z, please uncomment the line "if (index >= 9): index += 1" in the print section below. This should fix it for a 24 section output, removing J and Z. 

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
import cv2
from IPython.display import clear_output
import time
import canny
import matplotlib.pyplot as plt

cam_port = 0

cam = cv2.VideoCapture(cam_port)

workingDirectory = "../image_assets/"
imgPath = workingDirectory + 'frame256.png'
modelPath = '../models/final_restricted_letters.keras'

# Load the trained Keras model
model = load_model(modelPath)  # Replace with the path to your model file

valid, frame = cam.read()

frameSize = frame.shape
x = frameSize[1]
y = frameSize[0]

left = int((x - 256) / 2)
right = left + 256
top = int((y - 256) / 2)
bottom = top + 256


start_point = (left, top)    # top left corner of box
end_point = (right, bottom)      # bottom right corner of box
color = (0, 0, 255)         # red (B,G,R)
thickness = 2

# Function to preprocess the image
# Apply mean/std from training data
mean = 133.31247
std = 16.0731

def preprocess_image(img): # img is a np array (captured image)
    # Crop center 256x256 pixel array
    cropped = img[top:bottom, left:right]

    # Change to grayscale
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)

    # Scale the image to (128, 128)
    resized = cv2.resize(gray, (0,0), fx = 0.5, fy = 0.5)

    # Scale pixel values to [-1, 1]
    normalized = (resized - mean) / std

    processed_image = np.expand_dims(normalized, axis =(0, -1))
    return processed_image

counter = 0

while True:

    valid, frame = cam.read()   # frame holds the image object (np array)

    # rectangle is 256x256
    rectangleOverlay = cv2.rectangle(frame, start_point, end_point, color, thickness)
    cv2.imwrite(workingDirectory + "frameRect.png", rectangleOverlay)

    cv2.imshow("Current Frame", rectangleOverlay)

    if counter >= 25:
        counter = 0

        processed_image = preprocess_image(frame)

        prediction = model.predict(processed_image, batch_size=1)
        prediction = prediction[0]
        print("Shape of prediction[0]:", prediction[0].shape)


        print("--------------------------------------")
        clear_output(wait=True)  # Clears the current output

        maxx = 0
        maxIndex = -1

        print(prediction)

        # print 5 most likely classes:
        prediction_indices = np.argsort((prediction))[::-1]
        sorted_prediction = prediction[prediction_indices]

        letters = ['O', 'W', 'Y', 'C', 'L']
        for i in range(5):
            print(f"Predicted class: {letters[prediction_indices[i]]}, Probability: {sorted_prediction[i]}") 
        
        time.sleep(0.5)
        print("--------------------------------------")
        
    
    else:
        counter += 1

    if cv2.waitKey(1) == ord('q'):
        break


cam.release()
cv2.destroyAllWindows()

[0.1890823  0.3474793  0.3108528  0.08358636 0.06899925]
Predicted class: W, Probability: 0.34747931361198425
Predicted class: Y, Probability: 0.3108527958393097
Predicted class: O, Probability: 0.1890822947025299
Predicted class: C, Probability: 0.08358636498451233
Predicted class: L, Probability: 0.06899925321340561
--------------------------------------


KeyboardInterrupt: 